# Installation

In [ ]:
!pip install pandas
!pip install pandas-profiling
!pip install transformers
!pip install evaluate
!pip install sentencepiece
!pip install -U scikit-learn
!pip install accelerate
!pip install gradio

# Download the dataset

In [10]:
import os

# Create a directory for the downloaded dataset.
dataset_name = "wongnai-dataset"
os.makedirs(dataset_name, exist_ok=True) 

# Download the dataset from google drive.
!wget https://github.com/wongnai/wongnai-corpus/raw/master/review/review_dataset.zip

# Unzip the dataset.
!unzip review_dataset.zip -d wongnai-dataset # for linux
# !tar -xzvf review_dataset.zip -C wongnai-dataset # for windows

# Remove the zip file.
!rm review_dataset.zip
# Remove the unrelated __MACOSX folder.
!rm -r wongnai-dataset/__MACOSX

--2023-08-26 22:15:07--  https://github.com/wongnai/wongnai-corpus/raw/master/review/review_dataset.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wongnai/wongnai-corpus/master/review/review_dataset.zip [following]
--2023-08-26 22:15:07--  https://raw.githubusercontent.com/wongnai/wongnai-corpus/master/review/review_dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14338706 (14M) [application/zip]
Saving to: ‘review_dataset.zip’

review_dataset.zip  100%[===================>]  13.67M  16.8MB/s    in 0.8s    

2023-08-26 22:15:08 (16.8 MB/s) - ‘review_dataset.zi

# Data Preparation

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the dataset into a pandas dataframe.
df = pd.read_csv("wongnai-dataset/w_review_train.csv", 
    sep=";",
    names=["review", "rating"],
    header=None
)
# Remove duplicate rows.
df.drop_duplicates(inplace=True)
# Remove newline (\n) characters from reviews.
df["review"] = df["review"].str.replace('\n','') # remove \n
# Create label column from rating column.
df["label"] = df["rating"].map({1: 0, 2: 1, 3: 2, 4: 3, 5: 1})
# Drop rating column because we only want review and label columns.
df.drop("rating", axis=1, inplace=True)
# Sample 1000 rows for the sake of example.
df = df.sample(1000)
# Split the dataset into train and test sets.
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42) # Random state is just for reproducibility.
# Save the train and test sets to csv files.
train_df.to_csv("./train.csv", index=False) 
val_df.to_csv("./test.csv", index=False)

# Load Dataset

In [13]:
from datasets import load_dataset

# Load the dataset from the csv files.
dataset = load_dataset("csv", data_files={"train": "train.csv", "test" : "test.csv" })
dataset

/Users/nim/miniforge3/envs/wongnai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Extracting data files: 100%|██████████| 2/2 [00:00<00:00, 813.09it/s]
Generating train split: 800 examples [00:00, 19861.86 examples/s]
Generating test split: 200 examples [00:00, 28385.92 examples/s]


DatasetDict({
    train: Dataset({
        features: ['review', 'label'],
        num_rows: 800
    })
    test: Dataset({
        features: ['review', 'label'],
        num_rows: 200
    })
})

# Create Metrics

In [14]:
import evaluate
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# Load accuracy metric.
accuracy = evaluate.load("accuracy")

# Create a dictionary of rating labels and their corresponding ids.
id2label = {0: "Very poor (1) ", 1: "Poor (2)", 2: "Average (3)", 3: "Good (4)", 4: "Exellent (5)"}
label2id = {"Very poor (1)": 0, "Poor (2)": 1, "Average (3)": 2, "Good (4)": 3, "Exellent (5)": 4} 

# Load Tokenizer and Model

In [15]:
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification

pretrained_model_name = "airesearch/wangchanberta-base-att-spm-uncased"

# Load pretrained model
model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name, 
    num_labels=5, 
    id2label=id2label, 
    label2id=label2id
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name,
    use_fast=False, 
    model_max_length = 100 # The maximum length (in number of tokens) for the inputs to the transformer model
)


def tokenize(examples):
    """
    Function to convert the review texts to tokens.
    For example: "I like this food" -> [101, 146, 108, 114, 117, 110, 170, 102]
    """
    return tokenizer(examples["review"], truncation=True)

# Applying the tokenize function to the our dataset.
dataset = dataset.map(tokenize, batched=True)
train_dataset, test_dataset = dataset["train"], dataset["test"]

# Data collator is used to create batches of input data from the dataset.
# DataCollatorWithPadding will dynamically pad the tokens to the maximum length in that batch.
# More info on data collators: https://www.youtube.com/watch?v=-RPeakdlHYo
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length')

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 200/200 [00:00<00:00, 1684.12 examples/s]


In [16]:
# Let's take a look at our first sample in the training set.
print(train_dataset[0])

{'review': 'วันดี วันที่สมุนครบทีม (มีบางคนแซวว่า เด็กในฮาเร็ม 555)เกณฑ์พลมาที่แยกประตูน้ำ หัวมุมทางไป Platinamแต่ละหน่อ เอาใจลำบาก / มาแบบนี้ก็ดี จะได้เลือกกินกันเองไลน์อาหารมากพอควร ...ถูกใจที่หมูกรอบ หมูแดง เฉือนเสิร์ฟ สดๆ นุ่มมมมม ซะขอบอกว่า ---> ไลน์ของหวาน เพียบ', 'label': 2, 'input_ids': [5, 10, 125, 81, 2573, 22238, 1497, 230, 1241, 23, 1042, 7294, 27, 6345, 20, 901, 8738, 76, 14432, 29, 3177, 172, 26, 12, 818, 20031, 10, 303, 1399, 75, 28, 10, 3, 1663, 2657, 11804, 953, 13962, 10, 6708, 2036, 10, 101, 10, 26, 227, 2575, 10, 492, 309, 241, 3207, 2035, 319, 82, 20489, 4748, 4608, 12, 18502, 10, 20838, 10, 16444, 4858, 10, 10258, 10, 2145, 2139, 2139, 10, 964, 20626, 10, 8156, 1992, 10, 2035, 10385, 10, 5471, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


# Training model

In [ ]:
from transformers import TrainingArguments, Trainer

# Create a TrainingArguments object to configure how to train the model.
training_args = TrainingArguments(
    output_dir="Wongnai_classification", # Where to store the final model.
    learning_rate=0.00001,  # Set learning rate
    per_device_train_batch_size=1,  # Batch size for training.
    per_device_eval_batch_size=1, # Batch size for evaluation.
    num_train_epochs=10,  # Number of training epochs.
    weight_decay=0.01, 
    evaluation_strategy="epoch", # How often to evaluate the model.
    save_strategy="epoch", # How often to save the model.
    load_best_model_at_end=True, # Whether to load the best model at the end of training.
    push_to_hub=False, # Whether to upload the final model to the Huggingface Hub.
    report_to="none" # Whether to report logging to any services.
)

# Create a trainer to train the model.
trainer = Trainer( 
    model=model, 
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics, 
)

In [ ]:
# Run training
trainer.train()

# Inference

In [12]:
example_text = (
"ปกติมาทานบ่อยอยู่แล้ว วันนี้ลองมาทานก๋วยเตี๋ยวเจดู "
"รสชาติสู้แบบปกติไม่ได้เพราะปกติอร่อยมากๆ แต่พอเป็นแนวเจ " 
"รสชาติของเครื่องจึงตกลง แต่น้ำซุปยังอร่อยเหมือนเดิม ส่วนไอศครีมกะทิราดซุปข้าวโพด "
"อร่อยดี ทานไม่บ่อย แต่คราวนี้ลองดู ช่วยให้มื้อนี้ดูดีขึ้นเยอะเลย"
)

## Use pipeline to predict

In [ ]:
from transformers import pipeline

# Point to the directory where the model weight is stored.
model_dir = "Wongnai_classification/checkpoint-8000/"
# Create a pipeline to classify the sentiment of the input text.
classifier = pipeline("sentiment-analysis", model=model_dir)
# Let's try it out!
output = classifier(example_text)
print(output)

## Manually predict

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Point to the directory where the model weight is stored.
model_dir = "Wongnai_classification/checkpoint-8000/"

# Create model from our fine-tuned checkpoint.
model = AutoModelForSequenceClassification.from_pretrained(
    model_dir, 
    local_files_only=True # Look for the model in the local directory, not on the Huggingface Hub.
)
# Load tokenizer from our pretrained model.
tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name, 
    use_fast=False
)

# Tokenize the example text to input_ids.
input_ids = tokenizer(example_text, return_tensors="pt")

with torch.no_grad(): # Disable gradient calculation because we are not training
    logits = model(**input_ids).logits

# Get the most probable class for the input text.
predicted_class_id = logits.argmax().item()
# Get the rating label from the predicted rating
predicted_rating_label = model.config.id2label[predicted_class_id]


print(f"Input review: {example_text}\nPredicted rating label: \"{predicted_rating_label}\"")

# Wrap with gradio

For a nice interactive demo, we use gradio to wrap our model and make it easy to use.

In [ ]:
from gradio.components import Textbox, Label
from gradio import Interface

# Define a function that will process the input text from gradio widget.
def classify_text(text: str) -> str:
    """Classify the sentiment of the input text and return the predicted rating."""
    output = classifier(text)
    prediction = output[0]

    predicted_rating = prediction["label"]
    confidence_score = round(prediction["score"], 4)
    return predicted_rating, confidence_score


# Create a gradio interface.
rating_inferface = Interface(
    fn=classify_text,
    inputs=Textbox(label="Review"),
    outputs=[
        Label(label="Predicted rating"),
        Label(label="Confidence score"),
    ],

    title="Wongnai review rating prediction",
    description="Classify the sentiment of the review into the predicted rating with confidence score.",
)
rating_inferface.launch()